### A very simple example of call to *GISCO* web-services...

Let us run a very simple example using the `happyGISCO` module. One can refer anytime to the documentation available at this address:  http://happygisco.readthedocs.io.

In [1]:
import happygisco
from happygisco import services

GOOGLEMAPS help: https://github.com/googlemaps/google-maps-services-python
! GOOGLEPLACES package (https://github.com/slimkrazy/python-google-places) not loaded !
GEOPY help: http://geopy.readthedocs.io/en/latest/
! missing SIMPLEJSON package (https://pypi.python.org/pypi/simplejson/) <class 'ImportWarning'> !


We first create an instance of a `GISCO`-based service.

In [2]:
serv=services.GISCOService()

In the following, we will use this instance to perform various operations, calling everytim *GISCO* web-services. 

Let us simply define a geolocation by its common name:

In [3]:
whereamI = 'Luxembourg'

We could also have chosen to actually use: `wheramI = "Luxembourg, Luxembourg"`. That is up to us actually...and to the service capability to resolve toponame ambibuity.

One can already simply run the service to retrieve the geographical coordinates of the considered location, since the method `place2coord` can be used for that purpose:

In [4]:
coords = serv.place2coord(whereamI)
print(coords)

[[49.8158683, 6.1296751], [49.6047585, 6.142580146666669], [48.3777152, 4.2576617], [49.6112768, 6.129799]]


Hence, we got this way all the geolocations that *"potentially"* correspond to the name `whereamI`. We can actually decide to retrieve one location instead of all of them, *e.g.* the first one in the list (rather arbitrarly):

In [5]:
coord = serv.place2coord(whereamI, unique=True)
print(coord)

[49.8158683, 6.1296751]


Note that any time, we actually call the *GISCO* geoservice... there is **no caching** of the data fetched through the service!

What if we *"revert"* the geocoding service to try to identify the name of the location:

In [6]:
serv.coord2place(coord)

'Schieren, 7421, Luxembourg'

Hence, it does not exactly correspond to the input location name... not so surprising. What if we run the same service on the whole collection of all retrieved geographical coordinates:

In [7]:
names = serv.coord2place(coords)
print(names)

['Schieren, 7421, Luxembourg', "Rue d'Itzig, Luxembourg, 1118, Luxembourg", "Rue de l'Église, Bouy-Luxembourg, (Grand Est), 10220, France", 'Luxembourg, 2525, Luxembourg']


Hum... note that one of the places retrieved is actually...in France. That's the ambiguity related to the toponame. Note that this issue is actually not solved when passing the country as well:

In [8]:
serv.coord2place(serv.place2coord('Luxembourg, Luxembourg'))

['Schieren, 7421, Luxembourg',
 "Rue d'Itzig, Luxembourg, 1118, Luxembourg",
 "Rue de l'Église, Bouy-Luxembourg, (Grand Est), 10220, France",
 'Luxembourg, 2525, Luxembourg']

We can use the outputs of the *GISCO* service, `names` and `coords` data, for visualisation. For that purpose, we will use the module [`folium`](https://github.com/python-visualization/folium) that builds ontop of [`Leaflet.js`](https://leafletjs.com) and enables to render simple maps in notebooks.

In [9]:
import folium
_names = [n.replace("'"," ") for n in names] # some bug issue with folium when dealing with "'"
m = folium.Map(location=coord, tiles='OpenStreetMap', zoom_start=6)
[folium.Marker(coords[i], popup='<i>%s</i>' % _names[i]).add_to(m) for i in range(len(_names))]
m

Now you can zoom in/out on the map, check the location designatd by the marker....

We can further use the *GISCO* service to retrieve the *geometry* associated to the location:

In [10]:
serv.place2area(whereamI)

[{'geometry': {'coordinates': [6.1296751, 49.8158683], 'type': 'Point'},
  'properties': {'country': 'Luxembourg',
   'extent': [5.7357006, 50.1827724, 6.5312481, 49.4478539],
   'name': 'Luxembourg',
   'osm_id': 2171347,
   'osm_key': 'place',
   'osm_type': 'R',
   'osm_value': 'country'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [5.439918625407589, 49.963667], 'type': 'Point'},
  'properties': {'country': 'Belgium',
   'extent': [4.9684415, 50.430377, 6.0344254, 49.4969821],
   'name': 'Luxembourg',
   'osm_id': 1412581,
   'osm_key': 'boundary',
   'osm_type': 'R',
   'osm_value': 'administrative',
   'state': 'Wallonia'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [6.142580146666669, 49.6047585],
   'type': 'Point'},
  'properties': {'country': 'Luxembourg',
   'extent': [5.998434, 49.692307, 6.3096867, 49.516975],
   'name': 'Canton Luxembourg',
   'osm_id': 407824,
   'osm_key': 'place',
   'osm_type': 'R',
   'osm_value': 'county'},
  'type': 'Feature'},
 {

or we are lazy:

In [11]:
area = serv.place2area(whereamI, unique=True)
print(area)

{'geometry': {'coordinates': [6.1296751, 49.8158683], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 2171347, 'osm_type': 'R', 'extent': [5.7357006, 50.1827724, 6.5312481, 49.4478539], 'country': 'Luxembourg', 'osm_key': 'place', 'osm_value': 'country', 'name': 'Luxembourg'}}


We are more interested in retrieving/identifying the NUTS value, say at level 2, associated to the geolocation. We can use the method `findnuts` for that purpose:

In [12]:
serv.place2nuts(whereamI, level=2)

{'attributes': {'CNTR_CODE': 'LU',
  'LEVL_CODE': '2',
  'NAME_LATN': 'Luxembourg',
  'NUTS_ID': 'LU00',
  'NUTS_NAME': 'Luxembourg',
  'OBJECTID': '333',
  'SHRT_ENGL': 'Luxembourg'},
 'displayFieldName': 'NUTS_ID',
 'layerId': 2,
 'layerName': 'NUTS_2013',
 'value': 'LU00'}

or if we are actually interested in retrieving all the NUTS levels (from 0 to 3):

In [13]:
serv.place2nuts(whereamI)

[{'attributes': {'CNTR_CODE': 'LU',
   'LEVL_CODE': '0',
   'NAME_LATN': 'LUXEMBOURG',
   'NUTS_ID': 'LU',
   'NUTS_NAME': 'LUXEMBOURG',
   'OBJECTID': '20',
   'SHRT_ENGL': 'Luxembourg'},
  'displayFieldName': 'NUTS_ID',
  'layerId': 2,
  'layerName': 'NUTS_2013',
  'value': 'LU'},
 {'attributes': {'CNTR_CODE': 'LU',
   'LEVL_CODE': '1',
   'NAME_LATN': 'LUXEMBOURG',
   'NUTS_ID': 'LU0',
   'NUTS_NAME': 'LUXEMBOURG',
   'OBJECTID': '97',
   'SHRT_ENGL': 'Luxembourg'},
  'displayFieldName': 'NUTS_ID',
  'layerId': 2,
  'layerName': 'NUTS_2013',
  'value': 'LU0'},
 {'attributes': {'CNTR_CODE': 'LU',
   'LEVL_CODE': '2',
   'NAME_LATN': 'Luxembourg',
   'NUTS_ID': 'LU00',
   'NUTS_NAME': 'Luxembourg',
   'OBJECTID': '333',
   'SHRT_ENGL': 'Luxembourg'},
  'displayFieldName': 'NUTS_ID',
  'layerId': 2,
  'layerName': 'NUTS_2013',
  'value': 'LU00'},
 {'attributes': {'CNTR_CODE': 'LU',
   'LEVL_CODE': '3',
   'NAME_LATN': 'Luxembourg',
   'NUTS_ID': 'LU000',
   'NUTS_NAME': 'Luxembourg',
 

Similarly, we could have run the service on the already computed geographical coordinates:

In [14]:
serv.coord2nuts(coord, level=2)

{'attributes': {'CNTR_CODE': 'LU',
  'LEVL_CODE': '2',
  'NAME_LATN': 'Luxembourg',
  'NUTS_ID': 'LU00',
  'NUTS_NAME': 'Luxembourg',
  'OBJECTID': '333',
  'SHRT_ENGL': 'Luxembourg'},
 'displayFieldName': 'NUTS_ID',
 'layerId': 2,
 'layerName': 'NUTS_2013',
 'value': 'LU00'}

That's is folks!